In [26]:
import os
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
import trimesh
import random
import csv
from tqdm import tqdm
from torch.utils.data import Dataset, DataLoader, random_split
from torch.utils.data._utils.collate import default_collate


class MeshDataset(Dataset):
    def __init__(self, csv_file, mesh_dir, transform=None):
        """
        Args:
            csv_file (string): Path to the csv file with mesh filenames and labels
            mesh_dir (string): Directory with all the mesh files
            transform (callable, optional): Optional transform for the mesh data
        """
        self.data_frame = pd.read_csv(csv_file, header=None, names=['filename', 'label'])
        self.mesh_dir = mesh_dir
        self.transform = transform
    
    def __len__(self):
        return len(self.data_frame)
    
    def __getitem__(self, idx):
        if torch.is_tensor(idx):
            idx = idx.tolist()
        
        # Get mesh filename and label
        mesh_name = self.data_frame.iloc[idx, 0].strip()
        mesh_path = os.path.join(self.mesh_dir, mesh_name)
        label = self.data_frame.iloc[idx, 1]
        
        # Load the mesh
        mesh = trimesh.load(mesh_path)
        
        # Extract mesh features
        vertices = torch.FloatTensor(mesh.vertices)
        faces = torch.LongTensor(mesh.faces)
        
        sample = {
            'filename': mesh_name,
            'vertices': vertices,
            'faces': faces,
            'label': torch.tensor(label, dtype=torch.long),
            'num_vertices': vertices.shape[0],
            'num_faces': faces.shape[0]
        }
        
        if self.transform:
            sample = self.transform(sample)
            
        return sample


def mesh_collate_fn(batch):
    """
    Custom collate function for meshes with different sizes
    """
    # Get all keys from the first item in the batch
    keys = batch[0].keys()
    
    collated_batch = {}
    
    for key in keys:
        if key in ['vertices', 'faces']:
            # These are variable-sized tensors, so we just keep them as a list
            collated_batch[key] = [item[key] for item in batch]
        else:
            # For other items, use the default collation
            collated_batch[key] = default_collate([item[key] for item in batch])
            
    return collated_batch

In [ ]:
def set_seed(seed):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
    torch.use_deterministic_algorithms(True)

set_seed(22)

# 讀data
mesh_dir = "obj_files/"
dataset = MeshDataset(csv_file='labels.csv', mesh_dir=mesh_dir)

# 創建 DataLoader
Data_loader = DataLoader(dataset, batch_size=4, shuffle=False, num_workers=0, collate_fn=mesh_collate_fn)

# 定義 MeshClassifier
class MeshClassifier(nn.Module):
    def __init__(self, input_dim=3, num_classes=3):
        super(MeshClassifier, self).__init__()

        # filter
        self.filter = nn.Conv1d(input_dim, 32, kernel_size=3, padding=1)

        self.conv1 = nn.Conv1d(32, 64, 1)
        self.conv2 = nn.Conv1d(64, 128, 1)
        self.conv3 = nn.Conv1d(128, 256, 1)
        self.conv4 = nn.Conv1d(256, 512, 1)

        self.fc1 = nn.Linear(512, 128)
        self.fc2 = nn.Linear(128, num_classes)
        self.dropout = nn.Dropout(0)

    def forward(self, vertices_list):
        batch_features = []
        
        for vertices in vertices_list:
            x = vertices.T.unsqueeze(0) # (N, 3) -> (1, 3, N)
            
            x = F.relu(self.filter(x))
            x = F.relu(self.conv1(x))
            x = F.relu(self.conv2(x))
            x = F.relu(self.conv3(x))
            x = F.relu(self.conv4(x))
            x = torch.max(x, dim=2)[0]  # 全域最大池化 (1, 512)
            batch_features.append(x.squeeze(0))  # (512,)

        batch_features = torch.stack(batch_features)  # (batch_size, 512)
        x = F.relu(self.fc1(batch_features))
        x = self.dropout(x)
        x = self.fc2(x)
        return x  

# initialize
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = MeshClassifier().to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.0005)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=4, gamma=0.5)
criterion = nn.CrossEntropyLoss()

# training
num_epochs = 20
for epoch in range(num_epochs):
    model.train()
    total_train_loss = 0.0
    
    progress_bar = tqdm(Data_loader, desc=f"Epoch {epoch+1}/{num_epochs}")

    for batch in progress_bar:
        optimizer.zero_grad()
        
        vertices_list = [v.to(device) for v in batch['vertices']]
        labels = batch['label'].to(device)
        output = model(vertices_list)
        loss = criterion(output, labels)
        loss.backward()
        optimizer.step()
        
        total_train_loss += loss.item()

        # 更新 tqdm
        progress_bar.set_postfix(loss=loss.item())

    print(f"Epoch {epoch+1}/{num_epochs}, Training Loss: {total_train_loss:.3f}")

Epoch 1/20: 100%|██████████| 42/42 [00:14<00:00,  2.99it/s, loss=2.12]   


Epoch 1/20, Training Loss: 58.541


Epoch 2/20: 100%|██████████| 42/42 [00:15<00:00,  2.68it/s, loss=2.52] 


Epoch 2/20, Training Loss: 42.992


Epoch 3/20: 100%|██████████| 42/42 [00:19<00:00,  2.12it/s, loss=1.98] 


...

Epoch 18/20, Training Loss: 11.677


Epoch 19/20: 100%|██████████| 42/42 [00:17<00:00,  2.43it/s, loss=0.633]   


Epoch 19/20, Training Loss: 7.796


Epoch 20/20: 100%|██████████| 42/42 [00:18<00:00,  2.32it/s, loss=0.535]  

Epoch 20/20, Training Loss: 4.845


In [28]:
torch.save(model.state_dict(), "mesh_classifier.pth")

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model = MeshClassifier().to(device)
model.load_state_dict(torch.load("mesh_classifier.pth", map_location=device))
model.eval()  

def predict_and_save(test_files, output_csv, model, device):
    results = []

    for filename in os.listdir(test_files):
        if filename.endswith(".obj"):
            mesh_path = os.path.join(test_files, filename)

            # 加載 Mesh
            mesh = trimesh.load(mesh_path)
            vertices = torch.FloatTensor(mesh.vertices).unsqueeze(0).to(device)  # (N, 3) -> (1, N, 3)

            with torch.no_grad():
                output = model([vertices.squeeze(0)])  # 取得 logits
                probabilities = F.softmax(output, dim=1)  # 轉換為機率分佈

                if probabilities[0, 2] > 0.3: 
                    predicted_class = 2
                else:
                    predicted_class = torch.argmax(probabilities, dim=1).item()  # 找出機率最高的類別

            # 轉換為 Python 數值格式，並四捨五入到小數4位
            prob_list = [round(p, 4) for p in probabilities.squeeze(0).tolist()]
            results.append([filename, predicted_class, prob_list[0], prob_list[1], prob_list[2]])

            print(f"檔案: {filename}, 預測類別: {predicted_class}, 機率: {prob_list}")

    # 儲存結果至 CSV
    with open(output_csv, mode="w", newline="") as file:
        writer = csv.writer(file)
        writer.writerow(["filename", "predicted_class", "class_0_prob", "class_1_prob", "class_2_prob"])
        writer.writerows(results)

test_files = "obj_files/"
output_csv = "prediction.csv"
predict_and_save(test_files, output_csv, model, device)

檔案: chair_001_01.obj, 預測類別: 0, 機率: [0.9941, 0.005, 0.0009]
檔案: chair_001_02.obj, 預測類別: 0, 機率: [1.0, 0.0, 0.0]
檔案: chair_001_03.obj, 預測類別: 0, 機率: [0.9993, 0.0, 0.0007]
檔案: chair_001_04.obj, 預測類別: 0, 機率: [1.0, 0.0, 0.0]
檔案: chair_001_05.obj, 預測類別: 0, 機率: [1.0, 0.0, 0.0]
檔案: chair_001_06.obj, 預測類別: 0, 機率: [1.0, 0.0, 0.0]
檔案: chair_001_07.obj, 預測類別: 0, 機率: [0.9985, 0.0006, 0.0009]
檔案: chair_001_08.obj, 預測類別: 0, 機率: [0.9382, 0.0225, 0.0393]
檔案: chair_001_09.obj, 預測類別: 0, 機率: [0.9987, 0.0007, 0.0006]
檔案: chair_001_10.obj, 預測類別: 0, 機率: [1.0, 0.0, 0.0]
檔案: chair_001_11.obj, 預測類別: 0, 機率: [1.0, 0.0, 0.0]
檔案: chair_001_12.obj, 預測類別: 0, 機率: [1.0, 0.0, 0.0]
檔案: chair_001_13.obj, 預測類別: 0, 機率: [0.9999, 0.0001, 0.0]
檔案: chair_001_14.obj, 預測類別: 0, 機率: [0.9998, 0.0002, 0.0001]
檔案: chair_001_15.obj, 預測類別: 0, 機率: [0.9984, 0.0011, 0.0005]
檔案: chair_001_16.obj, 預測類別: 0, 機率: [0.9571, 0.0412, 0.0017]
檔案: chair_001_17.obj, 預測類別: 0, 機率: [0.9987, 0.0013, 0.0]
檔案: chair_001_18.obj, 預測類別: 0, 機率: [0.9942, 0.0057, 0